<a href="https://colab.research.google.com/github/fdsig/A-Lamp/blob/master/AVA_Image_Binary_Classification_aumentaions_mod_debuged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import shutil
import pandas as pd
import numpy as np
import zipfile
import random
from shutil import copyfile
import albumentations as A
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import io
import time
import os
import subprocess
import sys
from tqdm import tqdm
from torchvision import datasets, transforms
import torchvision
from PIL import Image
from torchvision.utils import save_image




In [3]:
def run_process(process = None, command = None):
    print(command)
    logname = process+'.log'
    env = os.environ.copy()
    with io.open(logname, 'wb') as writer, io.open(logname, 'rb', 1) as reader:
        process = subprocess.Popen(command, stdout=writer, shell=True, env=env)
        while process.poll() is None:
            sys.stdout.write(reader.read())
        # Read the remaining
        sys.stdout.write(reader.read())

## to innitialize 
depend = [ 
                'torch==1.7.0', 'torchvision==0.8.1', 'timm==0.3.2', 
                'timm==0.3.2', 'albumentations==0.4.6'
                ]
for dep in depend:
    run_process(command='pip install '+dep, process = dep.split('==')[0])

cleanup = True
if cleanup:
    current = [fid for fid in os.scandir() if 'drive' not in fid.name]
    for i in current:
        os.system('rm -rf '+i.name)


#!git clone https://github.com/fdsig/convit.git

#!git clone https://github.com/facebookresearch/convit.git
repositories = ['https://github.com/mawady/convit.git',
                ' https://github.com/fdsig/image_utils',
                'https://github.com/fdsig/A-Lamp']
for repo in repositories:
    run_process(command='git clone '+repo, process=repo.split('/')[-1])

convit_fids = [fid for fid in os.scandir('convit/')]
other_fids = [fid for fid in os.scandir('image_utils/')]
for fid in convit_fids+other_fids:
    shutil.move(fid.path,fid.name)
os.system(
    'rm convit && rm image_utils && rm -rf convit &&\
     rm -rf MPADA && rm -rf image_utils')

import image_getter
pull = image_getter.Get_Ava()
pull.parse_urls()
pull.google_getter()
pull.ava_txt()
pull.download_ava_files(own_drive=True, 
                        download=True, 
                        full=True, 
                        clear_current=False)


pip install torch==1.7.0
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
pip install torchvision==0.8.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
pip install timm==0.3.2
pip install timm==0.3.2
pip install albumentations==0.4.6
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65172 sha256=efa397832d46332f9155e817ba4ad33a2b04b493f1b93f56604eded93b0e3785
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    F

100%|██████████| 255508/255508 [00:10<00:00, 24323.66it/s]


0 already exist 
 all files = 255508


45it [00:00, 5199.69it/s]


255508 [<DirEntry 'images'>]


In [5]:

class Square:
    def __call__(self,img): 
            dims = np.array(img.size)
            m_x = dims[[dims.argmax()]][0]
            y_axis_pad = int(m_x-dims[1])//2 ;x_axis_pad = int(m_x-dims[0])//2
            y_axis_pad+=1;x_axis_pad+=1
            pad = (x_axis_pad, y_axis_pad, x_axis_pad, y_axis_pad)
            return torchvision.transforms.functional.pad(img,pad,0,'constant')

train_reflect_tansform = A.Compose([
            A.augmentations.transforms.LongestMaxSize(max_size=256),
            A.augmentations.transforms.PadIfNeeded(256,256,  always_apply=True),
            ]) 
train_transforms = transforms.Compose(
    [
        Square(),
     transforms.Resize((256,256)),
    ]
)


In [6]:
def meta_process():
    df = pd.read_csv('ava_meta_with_int_id_230721.csv')
    y_gt = df['mos_float'].values
    ids = df['ID'].values
    print(len(ids))
    y_gt_std, y_gt_mean = np.std(y_gt, axis = 0), np.mean(y_gt, axis = 0)
    exclude_below = y_gt_mean-y_gt_std*4
    exclude_above = y_gt_mean+y_gt_std*4
    ids = ids[np.argwhere(y_gt>=exclude_below)].ravel()
    y_gt = y_gt[np.argwhere(y_gt>=exclude_below)].ravel()
    print(len(y_gt))
    ids = ids[np.argwhere(y_gt<=exclude_above)].ravel()
    y_gt = y_gt[np.argwhere(y_gt<=exclude_above)].ravel()
    print(len(ids),len(y_gt))
    ids_low = ids[np.argwhere(y_gt<4.99)].ravel().astype(int)
    ids_high = ids[np.argwhere(y_gt>5.01)].ravel().astype(int)
    to_include = np.concatenate((ids_low,ids_high), axis=0)
    len(to_include)
    return df[df['ID'].isin(to_include)]

In [ ]:
class Pre_pipe:
    def __init__(self):
        
        self.subpatch=False, 
        self.move=False
        self.box_n = False
        self.src = 'Images/images/'
        self.dest = 'data/'
        self.miss = None
        #create desitination paths
        self.test_train = ['train/','test/', 'val/']
        self.classes = ['class_1/','class_0/']
        self.create_dict()
    
    def destination(self):
        for i in self.test_train:
            for j in self.classes:
                os.makedirs(self.dest+i+j, exist_ok=True)
                print(f'made dir = {self.dest+i+j}')
        
    def create_dict(self):
        self.destination()
        train_dir,test_dir,val_dir = self.test_train
        class_1, class_0 = self.classes 
        src_did = self.src
        df = meta_process()
        y_g = df[['threshold','ID','set']].to_dict('index')
        y_g_dict = {str(y_g[pair_key]['ID']):y_g[pair_key]for pair_key in y_g}
        y_g_dict = {key:{**y_g_dict[key],**{'src':src_did+key+'.jpg'}} for key in y_g_dict} 
        self.y_g_dict = {key:{**y_g_dict[key], **{'dest':class_1}} if y_g_dict[key]['threshold']==1 
                    else {**y_g_dict[key], **{'dest':class_0}} for key in y_g_dict }
        print(len(self.y_g_dict))
        

    def copy(self, subpatch=None, move=None):
        
        if not self.move and self.subpatch:
            print('exected')
            box = f'_patch_{self.box_n}{ext}'    
            if self.miss:
                miss_copy = (k 
                    for i in os.scandir('data/') 
                    for j in os.scandir(i) 
                    for k in os.scandir(j))
                for fid in tqdm(miss_copy):
                    shutil.move(fid.path,self.root+fid.name)
                
            ##some process
            for key in tqdm(y_g_dict):
                
                if y_g_dict[key]['set'] == 'training':
                    self.y_g_dict[key]['fid']=root+train_dir+y_g_dict[key]['dest']+key+box
                elif y_g_dict[key]['set'] == 'test':
                    self.y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']+key+box
                else:
                    self.y_g_dict[key]['fid']=root+val_dir+y_g_dict[key]['dest']+key+box
                
                if not os.path.isfile(self.y_g_dict[key]['fid']): 
                    print('exected')
                    src = y_g_dict[key]['src']
                    bbox = self.y_g_dict[key]['BBoxes'][self.box_n]
                    self.img = read(src)[bbox[1]:bbox[3], bbox[0]:bbox[2],:]

                    try:
                        cv2.imwrite(self.y_g_dict[key]['fid'],self.img)
                    except:
                        print(self.y_g_dict[key]['fid'], 'not saved')
    
    def augment(self,  augment_type=None):
        read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
        y_g_dict = self.y_g_dict
        root = self.dest
        ext = '.jpg'
        train_dir,test_dir,val_dir = self.test_train
        for key in tqdm(y_g_dict):
            
            if augment_type == 'zero_pad':
                img = read(y_g_dict[key]['src'])
                img = train_transforms(Image.fromarray(img))
                img = np.asarray(img).astype('uint8')
            elif augment_type == 'reflect':
                img = read(y_g_dict[key]['src'])
                img = train_reflect_tansform(image=img)['image']

            if y_g_dict[key]['set'] == 'training':
                y_g_dict[key]['fid']=root+train_dir+y_g_dict[key]['dest']+augment_type+key+ext
                cv2.imwrite(y_g_dict[key]['fid'],img)
            elif y_g_dict[key]['set'] == 'test':
                y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']+augment_type+key+ext
                cv2.imwrite(y_g_dict[key]['fid'],img)
            else:
                y_g_dict[key]['fid']=root+val_dir+y_g_dict[key]['dest']+augment_type+key+ext
                cv2.imwrite(y_g_dict[key]['fid'],img)
        
    def undo(self):
        fids = [k for i in os.scandir('data/') for j in os.scandir(i) for k in os.scandir(j)]
        dest = 'Images/images/'
        all_ims = len(list(os.scandir(dest)))
        print(f'there are {all_ims} ins Source')
        for i in fids:
            shutil.move(i.path,dest+i.name )
        fids = [k for i in os.scandir('data/') for j in os.scandir(i) for k in os.scandir(j)]
        all_ims = len(list(os.scandir(dest)))
        print(f'there are {all_ims} ins Source')
        

                
                
    def get_patches(self):
        self.subpach = True
        bbox_path = '/content/A-Lamp/multi_patch_subnet/adaptive_patch_selection/output-split'
        batches_dir = list(os.walk(bbox_path))[0][1]
        bbox_files = {i[-2:]:[j.path for j in os.scandir(bbox_path+'/'+i)] for i in batches_dir}
        keys = list(bbox_files.keys())
        for key in tqdm(keys):
            for fl in bbox_files[key]:
                with open(fl, 'rb') as pk:
                    if 'df' not in locals():
                        df = pd.read_pickle(pk)
                    else:
                        df = df.append(pd.read_pickle(pk))
                    
        boxes_dict = {
            k.split('/')[-1].split('.')[0]:{'BBoxes':v} 
            for k,v in zip(df['Filename'],df['BBoxes']) 
            }

        self.y_g_dict = {
            key:{**self.y_g_dict[key],**boxes_dict[key]} 
            for key in self.y_g_dict if key in boxes_dict
            }

pre = Pre_pipe()
pre_args = ['reflect','zero_pad']
pre_args = ['reflect','zero_pad']
model_args = ['--model convit_base']
args = ['python main.py', '--pretrained', '--batch-size 50' ,'--epochs 20', '--data-path data/', '--nb_classes 2', '--nb_classes_pre 1000']

for arg in pre_args:
    os.system('rm -rf data/')
    pre.destination()
    pre.move == True
    pre.miss == False
    print(f'Augmenting with: {arg}')
    pre.augment(augment_type=arg)
    for size in model_args:
        args.insert(4, '--output_dir drive/MyDrive/0.AVA/results/'+arg+size.split(' ')[-1])
        args.insert(3,size)
        command = ' '.join(args)
        print(f'Runing Process with Args : {command}')
        run_process(process='convit', command=command)
        args.pop(3);args.pop(4)

   
        



made dir = data/train/class_1/
made dir = data/train/class_0/
made dir = data/test/class_1/
made dir = data/test/class_0/
made dir = data/val/class_1/
made dir = data/val/class_0/
255502
255411
255403 255403
253030
made dir = data/train/class_1/
made dir = data/train/class_0/
made dir = data/test/class_1/
made dir = data/test/class_0/
made dir = data/val/class_1/
made dir = data/val/class_0/
Augmenting with: reflect


 25%|██▌       | 64446/253030 [10:28<26:44, 117.56it/s]

In [ ]:

for arg in pre_args:
    args[4]+=arg+'_'
    print(args)
    command = ' '.join(args)
    print(command)
        


NameError: ignored

In [ ]:
'''boxes = [0,1,2,3,4]
for box in boxes:
    os.system('rm -rf data/')
    print('data' in [i.name for i in os.scandir('.')])
    pre.box_n=box
    pre.destination()
    print('data' in [i.name for i in os.scandir('.')])
    pre.miss = True
    pre.copy()
    args[4]='--output_dir drive/MyDrive/0.AVA/results/patch_no_'+str(box)
    command = ' '.join(args)
    fid = list(os.scandir('data/test/class_0/'))[1].path
    img = cv2.imread(fid)
    plt.imshow(img)
    run_process(process = 'convit', command = command)'''

In [ ]:
'''read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
for key in tqdm(pre.y_g_dict):
    print(key)
    fid = pre.y_g_dict[key]['src']
    print(fid)

    if 'image' in pre.y_g_dict[key]:
        img = pre.y_g_dict[key]['image']
        cv2.imwrite(fid,img)
        pre.y_g_dict[key].pop('image')

    else:
        img = read(fid)
        zero_pad_array = reflect(img=Image.fromarray(img))
        save_image(zero_pad_array,fid)
'''

In [ ]:
 fid_ = pre.y_g_dict[list(pre.y_g_dict.keys())[5]]['src']
 read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
 img = read(fid_)
 zero_pad_array = train_transforms(Image.fromarray(img)).permute(1,2,0)
 plt.imshow(zero_pad_array)

In [ ]:
 fid_ = pre.y_g_dict[list(pre.y_g_dict.keys())[5]]['src']
 read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
 img = read(fid_)
 zero_pad_array = train_reflect_tansform(image=img)['image']
 plt.imshow(zero_pad_array)